In [22]:
from pathlib import Path

import pandas as pd

In [23]:
def transform(df: pd.DataFrame):
    """Transform the census data into a more usable format."""
    df = df.reset_index().melt(id_vars="index")
    df = df.rename(
        columns={
            "index": "Education",
            "variable_0": "Age Group"
        }
    )
    df = df.drop(df[df["Education"].isna()].index)

    df["Education"] = df["Education"].str.strip(". ")

    MARITAL_GROUPS = (
        "All Marital Classes", # found in 2016
        "All Marital Statuses",
        "Women Ever Married",
        "Women Never Married"
    )
    marital_group = None
    marital_statuses = []
    for i, row in df.iterrows():
        if row["Education"] in MARITAL_GROUPS:
            marital_group = row["Education"]
        marital_statuses.append(marital_group)
    df["Marital Status"] = marital_statuses
    # Drop the rows that only delimited the beginning of a martial status group
    df = df.filter(df[df["Education"] != df["Marital Status"]].index, axis=0)
    # Drop those initial rows that won't fall into a marital status group
    df = df.filter(df[df["Marital Status"].notna()].index, axis=0)
    # Drop the section header row
    df = df.filter(df[df["Education"] != "EDUCATIONAL ATTAINMENT"].index, axis=0)
    
    df["variable_1"] = df["variable_1"].str.strip()
    return df.pivot(index=["Education", "Age Group", "Marital Status"], columns="variable_1", values="value")

def pre_clean_2012(df: pd.DataFrame):
    """Clean up the 2012 census sheet."""
    # Drop empty columns
    df = df.dropna(axis=1, how="all")

    # Build corrected column multiindex
    multi_index = []
    last_age_group = None
    for age_group, metric in list(df.columns):
        if age_group.startswith("Unnamed: "):
            age_group = last_age_group
        else:
            last_age_group = age_group
        if metric.startswith("Unnamed: "):
            continue
        multi_index.append((age_group, metric.strip()))
    idx = pd.MultiIndex.from_tuples(multi_index)
    df.columns = idx
    return df


dataframes = []
# Read in files
for path in Path("data/raw").iterdir():
    print(path)
    if path.suffix not in (".xlsx", ".csv"):
        continue
    try:
        # Filenames should be formatted '... - YYYY.ext'
        year = int(path.with_suffix('').name.split(" - ")[-1])
        if year == 2012:
            # 2012 has some odd formatting going on, need to do a bit more
            # cleaning beforehand
            ydf = pd.read_excel(
                path,
                skiprows=7,
                header=[0,1],
                index_col=0,
                nrows=25
            )
            ydf = pre_clean_2012(ydf)
        else:
            if path.suffix == ".xlsx":
                ydf = pd.read_excel(
                    path,
                    skiprows=6,
                    header=[0,1],
                    index_col=0,
                    nrows=28,
                    engine="openpyxl"
                )
            else:
                print(f"Skipping '{path!s}'")
                continue
        ydf = transform(ydf)
        ydf["Year"] = year
        dataframes.append(ydf)
    except:
        raise
        print(f"Failed '{path!s}'")

_df = pd.concat(dataframes).reset_index(drop=False)
_df

data/raw/t3c - 2014.xlsx
data/raw/t3c - 2018.xlsx
data/raw/.DS_Store
data/raw/t3d - 2012.xlsx
data/raw/t3c - 2016.xlsx
data/raw/t3c - 2020.xlsx


variable_1,Education,Age Group,Marital Status,"Children ever born per 1,000 women",Percent childless,Total women,Year
0,ALL WOMEN,Women 15 to 19 years old,Women Never Married,55,95.9,10202,2014
1,ALL WOMEN,Women 15 to 44 years old,Women Never Married,1126.0,47.6,62683.0,2014
2,ALL WOMEN,Women 15 to 50 years old,Women Never Married,1270.0,42.4,NaN,2014
3,ALL WOMEN,Women 20 to 29 years old,Women Never Married,656.0,62.6,21704.0,2014
4,ALL WOMEN,Women 30 to 39 years old,Women Never Married,1709.0,23.9,20476.0,2014
...,...,...,...,...,...,...,...
565,"Some college, no degree",Women 30 to 39 years,Women Ever Married,2012.0,14.7,2122.0,2020
566,"Some college, no degree",Women 30 to 39 years,Women Never Married,1092.0,45.8,1108.0,2020
567,"Some college, no degree",Women 40 to 50 years,All Marital Statuses,2138.0,15.3,3023.0,2020
568,"Some college, no degree",Women 40 to 50 years,Women Ever Married,2305.0,10.5,2533.0,2020


In [24]:
df = _df.copy()

In [25]:
# Clean up whitespace and unusual decimals
df["Age Group"] = df["Age Group"].str.strip(" .")

age_group_to_range = {
    'Women 15 to 19 years1': '15 - 19',
    'Women 15 to 44 years': '15 - 44',
    'Women 15 to 50 years': '15 - 50',
    'Women 20 to 29 years': '20 - 29',
    'Women 30 to 39 years': '30 - 39',
    'Women 40 to 50 years': '40 - 50',
    'Women 15 to 19 years old': '15 - 19',
    'Women 15 to 44 years old': '15 - 44',
    'Women 15 to 50 years old': '15 - 50',
    'Women 20 to 29 years old': '20 - 29',
    'Women 30 to 39 years old': '30 - 39',
    'Women 40 to 50 years old': '40 - 50'
}
df["Age Group"] = df["Age Group"].replace(age_group_to_range)
df

variable_1,Education,Age Group,Marital Status,"Children ever born per 1,000 women",Percent childless,Total women,Year
0,ALL WOMEN,15 - 19,Women Never Married,55,95.9,10202,2014
1,ALL WOMEN,15 - 44,Women Never Married,1126.0,47.6,62683.0,2014
2,ALL WOMEN,15 - 50,Women Never Married,1270.0,42.4,NaN,2014
3,ALL WOMEN,20 - 29,Women Never Married,656.0,62.6,21704.0,2014
4,ALL WOMEN,30 - 39,Women Never Married,1709.0,23.9,20476.0,2014
...,...,...,...,...,...,...,...
565,"Some college, no degree",30 - 39,Women Ever Married,2012.0,14.7,2122.0,2020
566,"Some college, no degree",30 - 39,Women Never Married,1092.0,45.8,1108.0,2020
567,"Some college, no degree",40 - 50,All Marital Statuses,2138.0,15.3,3023.0,2020
568,"Some college, no degree",40 - 50,Women Ever Married,2305.0,10.5,2533.0,2020


In [26]:
df["Marital Status"] = df["Marital Status"].replace("All Marital Classes", "All Marital Statuses")

# Filter for all marital statuses
df = df.filter(df[df["Marital Status"] == "All Marital Statuses"].index, axis=0)
df

variable_1,Education,Age Group,Marital Status,"Children ever born per 1,000 women",Percent childless,Total women,Year
6,Associate's degree,15 - 19,All Marital Statuses,113,93.2,62,2014
9,Associate's degree,15 - 44,All Marital Statuses,1388.0,35.2,5800.0,2014
12,Associate's degree,15 - 50,All Marital Statuses,1515.0,31.0,7297.0,2014
15,Associate's degree,20 - 29,All Marital Statuses,717.0,57.5,2197.0,2014
18,Associate's degree,30 - 39,All Marital Statuses,1746.0,23.3,2398.0,2014
...,...,...,...,...,...,...,...
555,"Some college, no degree",15 - 44,All Marital Statuses,922.0,57.2,11590.0,2020
558,"Some college, no degree",15 - 50,All Marital Statuses,1070.0,51.8,13270.0,2020
561,"Some college, no degree",20 - 29,All Marital Statuses,383.0,75.7,5737.0,2020
564,"Some college, no degree",30 - 39,All Marital Statuses,1696.0,25.4,3230.0,2020


In [27]:
# Clean up placeholder values

# Z = Represents zero or rounds to zero.
# N = Not available.
# D = Suppressed for disclosure avoidance.
df[["Children ever born per 1,000 women", "Percent childless", "Total women"]] = df[["Children ever born per 1,000 women", "Percent childless", "Total women"]]\
    .astype(str)\
    .apply(lambda s: s.str.strip().str.replace(',', ''))\
    .replace({"Z": '0.0', "D": "nan", "N": "nan"})\
    .astype(float)

df

variable_1,Education,Age Group,Marital Status,"Children ever born per 1,000 women",Percent childless,Total women,Year
6,Associate's degree,15 - 19,All Marital Statuses,113.0,93.2,62.0,2014
9,Associate's degree,15 - 44,All Marital Statuses,1388.0,35.2,5800.0,2014
12,Associate's degree,15 - 50,All Marital Statuses,1515.0,31.0,7297.0,2014
15,Associate's degree,20 - 29,All Marital Statuses,717.0,57.5,2197.0,2014
18,Associate's degree,30 - 39,All Marital Statuses,1746.0,23.3,2398.0,2014
...,...,...,...,...,...,...,...
555,"Some college, no degree",15 - 44,All Marital Statuses,922.0,57.2,11590.0,2020
558,"Some college, no degree",15 - 50,All Marital Statuses,1070.0,51.8,13270.0,2020
561,"Some college, no degree",20 - 29,All Marital Statuses,383.0,75.7,5737.0,2020
564,"Some college, no degree",30 - 39,All Marital Statuses,1696.0,25.4,3230.0,2020


In [28]:
# Convert to more useful units

df["Children Ever Born per Woman"] = df["Children ever born per 1,000 women"] / 1000
# 'Total women' is in thousands
df["Total Women"] = df["Total women"] * 1000

df["Childlessness"] = df["Percent childless"] / 100

df = df.drop(columns=["Children ever born per 1,000 women", "Percent childless", "Total women"])
df

variable_1,Education,Age Group,Marital Status,Year,Children Ever Born per Woman,Total Women,Childlessness
6,Associate's degree,15 - 19,All Marital Statuses,2014,0.113,62000.0,0.932
9,Associate's degree,15 - 44,All Marital Statuses,2014,1.388,5800000.0,0.352
12,Associate's degree,15 - 50,All Marital Statuses,2014,1.515,7297000.0,0.310
15,Associate's degree,20 - 29,All Marital Statuses,2014,0.717,2197000.0,0.575
18,Associate's degree,30 - 39,All Marital Statuses,2014,1.746,2398000.0,0.233
...,...,...,...,...,...,...,...
555,"Some college, no degree",15 - 44,All Marital Statuses,2020,0.922,11590000.0,0.572
558,"Some college, no degree",15 - 50,All Marital Statuses,2020,1.070,13270000.0,0.518
561,"Some college, no degree",20 - 29,All Marital Statuses,2020,0.383,5737000.0,0.757
564,"Some college, no degree",30 - 39,All Marital Statuses,2020,1.696,3230000.0,0.254


In [29]:
# Get rid of the name from the multiindex
df.columns.name = ""
df.insert(0, "Year", df.pop("Year"))

# Sort
df = df.sort_values(["Year", "Age Group", "Education"])\
    .reset_index(drop=True)

df

,Year,Education,Age Group,Marital Status,Children Ever Born per Woman,Total Women,Childlessness
0,2012,Associate's degree,15 - 19,All Marital Statuses,0.000,63000.0,1.000
1,2012,Bachelor's degree,15 - 19,All Marital Statuses,0.047,28000.0,0.953
2,2012,Graduate or professional degree,15 - 19,All Marital Statuses,0.524,7000.0,0.476
3,2012,High school graduate,15 - 19,All Marital Statuses,0.166,1481000.0,0.860
4,2012,Not a high school graduate,15 - 19,All Marital Statuses,0.045,7509000.0,0.967
...,...,...,...,...,...,...,...
175,2020,Bachelor's degree,40 - 50,All Marital Statuses,1.863,5899000.0,0.190
176,2020,Graduate or professional degree,40 - 50,All Marital Statuses,1.702,4196000.0,0.192
177,2020,High school graduate,40 - 50,All Marital Statuses,2.092,4869000.0,0.134
178,2020,Not a high school graduate,40 - 50,All Marital Statuses,2.676,1810000.0,0.077


In [30]:
df.to_csv("data/clean/biannual_census_2012-2020.csv", index=False)